In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager
import numpy as np
import pandas as pd
from janome.tokenizer import Tokenizer
from collections import Counter
import datetime as dt
import sys
import codecs
import time
import pickle
import os
import csv
import io

def coding_log():
	"""
	このコードの変更、追加、改善の記録をこの関数で管理する。
	「coding_log.__doc__」と入力することで関数内のコメントが呼び出せる。
	記載内容は「日付：変更・追加した関数名：変更内容」で記録
	-------------------------------------------------------
    2019/07/08 jupyter環境下で動作できるように修正。ただし、テキストのみ対応。次はpandasで入出力できるように検討

	-------------------------------------------------------
	"""

def document():
	"""
	このファイルの内容を記載し、ドキュメントとします。
	記載内容の構成は以下になります。
	====================================
	
	====================================
	
	"""

cps = []

class CorpusElement:
	def __init__(self, text = '', tokens = [], pn_scores = []):
		self.text = text
		self.tokens = tokens
		self.pn_scores = pn_scores

def ja_tokenize(text,types):
	ja_tokenizer = Tokenizer()
	res = []
	lines = text.split()
	lines = lines[1:]
	for line in lines:
		malist = ja_tokenizer.tokenize(line)
		for tok in malist:
			ps = tok.part_of_speech.split(",")[0]
			if not ps in [types]: continue
			w = tok.base_form
			if w == "*" or w == "": w = tok.surface
			if w == "" or w == "\n": continue
			res.append(w)
	return res

def dict_list(text):
	ja_tokenizer = Tokenizer()
	lines = text.split()
	diclist = []
	for text in lines:
		tokens = ja_tokenizer.tokenize(text)
		element = CorpusElement(text, tokens)
		cps.append(element)

def readFile(filename,ench):
    try:
        with codecs.open(filename, "r", ench, "ignore") as file:
            df = file.read()
            return df
    except IOError as e:
        sys.exit("Unable to open file: {}".format(e))

def output(data,filename):
	with codecs.open(filename + '.csv', 'w','cp932','ignore') as f:
		f.write('コメント,マネポジ値\n')
		for key, value in data.items():
			f.write('{},{}\n'.format(key,value))
	f.close()	

def output2(data,filename,dirname):
	with codecs.open(dirname + filename + '.csv', 'w','cp932','ignore') as f:
		csvWriter = csv.writer(f, lineterminator='\n')
		csvWriter.writerows(data.most_common())
	f.close()

def count(col,data):
	"""頻出単語をカウントする関数"""
	lists = []
	for i in range(len(data)-1):
		lists.append(data[i+1][col])
	counter = Counter(lists)
	return counter

def csvReader(filename,ench,delim = ','):		
	try:
		with codecs.open(filename, "r", ench, "ignore") as file:
			df = pd.read_csv(file, delimiter= delim)
			return df
	except IOError as e:
		sys.exit("Unable to open file: {}".format(e))

def load_pn_dict():
	dic = {}

	with codecs.open('pn/pn_ja.txt', 'r', 'shift-jis') as f:
		lines = f.readlines()
		for line in lines:
			columns = line.split(':')
			dic[columns[0]] = float(columns[3])

	return dic

def get_pn_scores(tokens, pn_dic):
	scores = []
	for surface in [t.surface for t in tokens if t.part_of_speech.split(',')[0] in ['動詞','名詞','形容詞','副詞']]:
		if surface in pn_dic:
			scores.append(pn_dic[surface])
	return scores

fname = 'CallData201712.csv'

if __name__ == "__main__":
    """
    処理
    """
    save_name1 = '/names_'+ fname
    save_name2 = '/sem_'+ fname
    types1 = '名詞'
    types2 = '形容詞'
    outname = os.path.abspath('.') + '/output_' + fname
    data1 = readFile(fname,'cp932')
#     data1 = str(data1.values.flatten())
    ##単語カウント#####
    token = ja_tokenize(data1,types1)
    token2 = ja_tokenize(data1,types2)
    counter = Counter(token)
    counter2 = Counter(token2)
    output2(counter,save_name1,os.path.abspath('.'))
    output2(counter2,save_name2,os.path.abspath('.'))
    ################

    ##ネガポジ値算出##
    outdic = {}

    # with codecs.open("pn/pn_ja.txt", "r", "shift-jis", "ignore") as file:
    # df = pd.read_csv(file, sep = ":",names = ('Word', ' Reading', 'POS', 'PN'))

    dict_list(data1)
    pn_dic = load_pn_dict()
    
    i = 0
    for element in cps:
        element.pn_scores = get_pn_scores(element.tokens, pn_dic)
        if len(cps[i].pn_scores) != 0:
            outdic[io.StringIO(element.text).readline()] = float(sum(element.pn_scores)/float(len(cps[i].pn_scores)))
        i += 1

    output(outdic,outname)
    ################

In [2]:
outdic

{'受付日,管理番号,案件番号,受付時刻,曜日,受付担当者,件名,手段,処理1,処理2,進捗,完了日,地域ｺｰﾄﾞ１,性別1,年代1,車種名称,年式,対象,走行距離,販社名称,拠点名称,大分類,中小分類1,中小分類2,中小分類3,中小分類4,中小分類5,中小分類6,中小分類7,中小分類8,中小分類9,部位ｺｰﾄﾞ１,部位ｺｰﾄﾞ２,部位ｺｰﾄﾞ３,ＣＳｺｰﾄﾞ１,ＣＳｺｰﾄﾞ２,ＣＳｺｰﾄﾞ３,幹部報告,新中区分,登録日,情報提供1,車種区分,車台番号,車種名称1,車種名称2,車種名称3,車種名称4,相談分類,ｷｰﾜｰﾄﾞ１,ｷｰﾜｰﾄﾞ２,ｷｰﾜｰﾄﾞ３,ｴﾝｼﾞﾝ型式,顧客ｺｰﾄﾞ１,確認1,確認2,確認3,対象国,ﾃﾞｨｽﾄﾘﾋﾞｭｰﾀ,言語,地域,関連文書,仕様,海外登録番号,海外ｷｰﾜｰﾄﾞ１,海外ｷｰﾜｰﾄﾞ２,ﾄﾊﾞｷｰﾜｰﾄﾞ１,ﾄﾊﾞｷｰﾜｰﾄﾞ２,ﾄﾊﾞｷｰﾜｰﾄﾞ３,ﾄﾊﾞｷｰﾜｰﾄﾞ４,内容,更新日付': -0.4956957604938275,
 '2017-12-04,20171204B9099001,1.0,105349.0,月,榎本佑紀,最長１０年１０万ｋｍ特別保証延長／新車購入時の条件・保証延長点検実施店舗等照会,電話－FD,即答（保留なし）,自己解決,完了,20171204.0,,男性,４０代,車種不明(),,購入検討中,,,,問合せ,[契約・手続・費用]保証書関連手続,,,,,,,,,,,,,,,,,,本人,車種に無関係,,車種不明(),,,,国内乗用車,,,,,,CCWEB/その他,CBサーチ,,,,,,,,,,,,,,,［内容］三菱販社ではなく知人の業者からの新車購入を検討中(※車種名はご教示頂けず)。三菱では最長10年10万km特別保証延長を実施していると聞いた。10年10万kmでは5年目以降に販社にて点検を受ける必要がある事は把握している。10年10万km特別保証延長について確認したい。①新車購入時に何か条件はあるか？②自分は転勤族の為、車両購入時と5年経過時で在住地域が異なる可能性が高い。': -0.43215553068181817,
 '車両購入店と5年経過時の保証点検実施店は別店舗でも問題ないか？5年経過時の': -0.6235783333333333,
 '保証延長点検も購入店舗で受けなければいけないのか？③保

In [3]:
outname

'D:\\jupyter_home\\TextMining/output_CallData201712.csv'

In [3]:
outdic.keys()

dict_keys(['受付日,管理番号,案件番号,受付時刻,曜日,受付担当者,件名,手段,処理1,処理2,進捗,完了日,地域ｺｰﾄﾞ１,性別1,年代1,車種名称,年式,対象,走行距離,販社名称,拠点名称,大分類,中小分類1,中小分類2,中小分類3,中小分類4,中小分類5,中小分類6,中小分類7,中小分類8,中小分類9,部位ｺｰﾄﾞ１,部位ｺｰﾄﾞ２,部位ｺｰﾄﾞ３,ＣＳｺｰﾄﾞ１,ＣＳｺｰﾄﾞ２,ＣＳｺｰﾄﾞ３,幹部報告,新中区分,登録日,情報提供1,車種区分,車台番号,車種名称1,車種名称2,車種名称3,車種名称4,相談分類,ｷｰﾜｰﾄﾞ１,ｷｰﾜｰﾄﾞ２,ｷｰﾜｰﾄﾞ３,ｴﾝｼﾞﾝ型式,顧客ｺｰﾄﾞ１,確認1,確認2,確認3,対象国,ﾃﾞｨｽﾄﾘﾋﾞｭｰﾀ,言語,地域,関連文書,仕様,海外登録番号,海外ｷｰﾜｰﾄﾞ１,海外ｷｰﾜｰﾄﾞ２,ﾄﾊﾞｷｰﾜｰﾄﾞ１,ﾄﾊﾞｷｰﾜｰﾄﾞ２,ﾄﾊﾞｷｰﾜｰﾄﾞ３,ﾄﾊﾞｷｰﾜｰﾄﾞ４,内容,更新日付', '2017-12-04,20171204B9099001,1.0,105349.0,月,榎本佑紀,最長１０年１０万ｋｍ特別保証延長／新車購入時の条件・保証延長点検実施店舗等照会,電話－FD,即答（保留なし）,自己解決,完了,20171204.0,,男性,４０代,車種不明(),,購入検討中,,,,問合せ,[契約・手続・費用]保証書関連手続,,,,,,,,,,,,,,,,,,本人,車種に無関係,,車種不明(),,,,国内乗用車,,,,,,CCWEB/その他,CBサーチ,,,,,,,,,,,,,,,［内容］三菱販社ではなく知人の業者からの新車購入を検討中(※車種名はご教示頂けず)。三菱では最長10年10万km特別保証延長を実施していると聞いた。10年10万kmでは5年目以降に販社にて点検を受ける必要がある事は把握している。10年10万km特別保証延長について確認したい。①新車購入時に何か条件はあるか？②自分は転勤族の為、車両購入時と5年経過時で在住地域が異なる可能性が高い。', '車両購入店と5年経過時の保証点検実施店は別店舗でも問題ないか？5年経過時の', '保証延長点検も購入店舗で受けなければいけないのか？③保証延長点検の記録はどこに記載されるのか？④③以外に必要なものはあるか？⑤自分は転勤族の為、転勤のタイミングによって

In [4]:
outdic.values()

dict_values([-0.4956957604938275, -0.43215553068181817, -0.6235783333333333, -0.4021282782608695, -0.5037457272727273, -0.5777909130434782, -0.45857672727272725, -0.3374399090909091, -0.45906118749999997, -0.6115747333333332, -0.48481438461538473, -0.3396632645161291, -0.4401957999999999, -0.4551568988636363, -0.3968276153846154, -0.5469901756756759, -0.3089443, -0.48704160181818157, -0.3867260285714286, -0.66305275, -0.4990655, -0.5029002499999999, -0.7164886666666668, -0.6172005294117648, -0.557438, -0.6655276000000001, -0.3851349285714285, -0.4115692, -0.524571, -0.5520368749999999, -0.509558, -0.4760546428571429, -0.5088759666666667, -0.46015358571428566, -0.5536173750000001, -0.4676060625, -0.4747565, -0.7017135, -0.656787, -0.736071, -0.612145, -0.4916085, -0.612145, -0.5260360952380952, -0.5314847199999999, -0.5280246470588236, -0.4842951200000001, -0.5166184214285713, -0.5953550769230769, -0.6191794666666668, -0.4229551142857143, -0.5245264999999999, -0.6775228888888889, -0.602

In [5]:
pn_dic

{'優れる': 1.0,
 '良い': 0.999995,
 '喜ぶ': 0.999979,
 '褒める': 0.999979,
 'めでたい': 0.999645,
 '賢い': 0.999486,
 '善い': 0.999314,
 '適す': 0.999295,
 '天晴': 0.999267,
 '祝う': 0.999122,
 '功績': 0.999104,
 '賞': 0.998943,
 '嬉しい': 0.998871,
 '喜び': 0.998861,
 '才知': 0.998771,
 '徳': 0.998745,
 '才能': 0.998699,
 '素晴らしい': 0.998617,
 '芳しい': 0.998578,
 '称える': 0.998576,
 '適切': 0.998406,
 '崇める': 0.998357,
 '助ける': 0.990702,
 '抜きんでる': 0.998356,
 '清水': 0.98213,
 '雄雄しい': 0.998272,
 '仕合せ': 0.998208,
 '幸い': 0.997862,
 '吉兆': 0.997795,
 '秀でる': 0.997777,
 '宜しい': 0.997517,
 'きりりしゃんと': 0.997511,
 '矜持': 0.988946,
 '香しい': 0.997438,
 'しこなす': 0.997408,
 '遣りこなす': 0.997406,
 '優麗': 0.997337,
 '気高い': 0.997325,
 '手柄': 0.997308,
 '正義': 0.997278,
 '麗しい': 0.99726,
 '名高い': 0.997259,
 '感心': 0.997201,
 '誉れ': 0.997198,
 '卓説': 0.997194,
 '巍然': 0.997192,
 '崇敬': 0.997192,
 '美徳': 0.997143,
 '素早い': 0.997105,
 '背負ってる': 0.997074,
 '人材': 0.997049,
 '果報者': 0.997027,
 '多祥': 0.997027,
 '勇ましい': 0.996987,
 '秀逸': 0.996946,
 'まぐれ幸い': 0.996941,
 '賞揚': 0.9969